In [12]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x * x * y + y + 2


In [13]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)

print(result)

sess.close()

42


In [14]:
init = tf.global_variables_initializer()

with tf.Session() as see:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


In [15]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()

print(result)

sess.close()

42


In [16]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [17]:
tf.reset_default_graph()

In [18]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

m, n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

XT = tf.transpose(X)

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print (theta_value)

[[-3.7465141e+01]
 [ 4.3573415e-01]
 [ 9.3382923e-03]
 [-1.0662201e-01]
 [ 6.4410698e-01]
 [-4.2513184e-06]
 [-3.7732250e-03]
 [-4.2664889e-01]
 [-4.4051403e-01]]


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediticions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
        


Epoch:  0 MSE =  6.001269
Epoch:  100 MSE =  0.7972658
Epoch:  200 MSE =  0.6486859
Epoch:  300 MSE =  0.6146767
Epoch:  400 MSE =  0.592166
Epoch:  500 MSE =  0.575599
Epoch:  600 MSE =  0.5632662
Epoch:  700 MSE =  0.5540445
Epoch:  800 MSE =  0.54712063
Epoch:  900 MSE =  0.5418996


In [20]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

In [21]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediticions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [22]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

In [23]:
best_theta

array([[ 2.0701346 ],
       [ 0.8322433 ],
       [ 0.11718353],
       [-0.25360966],
       [ 0.33584756],
       [ 0.0031159 ],
       [-0.01102607],
       [-0.9062839 ],
       [-0.8712016 ]], dtype=float32)

In [24]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

In [25]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="prediticions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch =  0 MSE =  12.408015
Epoch =  100 MSE =  0.75519705
Epoch =  200 MSE =  0.5420873
Epoch =  300 MSE =  0.53317
Epoch =  400 MSE =  0.53053814
Epoch =  500 MSE =  0.5287964
Epoch =  600 MSE =  0.52754897
Epoch =  700 MSE =  0.5266498
Epoch =  800 MSE =  0.5260011
Epoch =  900 MSE =  0.5255332


In [26]:
best_theta

array([[ 2.0685523e+00],
       [ 8.1063598e-01],
       [ 1.2685774e-01],
       [-2.0784084e-01],
       [ 2.4839848e-01],
       [-1.3083928e-03],
       [-3.9607048e-02],
       [-8.5861272e-01],
       [-8.2600278e-01]], dtype=float32)

In [27]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [28]:
np.allclose(best_theta, best_theta_restored)

True

In [29]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

In [30]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir ="{}/run-{}/".format(root_logdir, now)

In [31]:
n_epochs = 1000
learning_rate = 0.01
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [32]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [33]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))


In [34]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

In [35]:
file_writer.close()

In [36]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255152],
       [-0.31211877],
       [ 0.38510376],
       [ 0.00434168],
       [-0.0123295 ],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)